In [1]:
# Prelude: import packages and set up the plotting.
%matplotlib widget

import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib
import matplotlib.pyplot as plt

import pyscf
import pyscf.gto
import pyscf.dft
import pyscf.tools.cubegen
import ase
import ase.units
import nglview

import IPython.display as disp

/usr/local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


Set up a DFT calculation using LDA and STO-3G for ethylene using the following distances and angles

In [2]:
dCC = 1.336
dCH = 1.085
aHCH = 116.58

In [3]:
# Compute the H-C-C angle based on the H-C-H angle.
aHCC = 180. - aHCH / 2.

In [4]:
# Build the geometry. Instead of Cartesian coordinates, we use a notation based
# on distances, angles and dihedrals, known as the Z matrix.
# The first line places the first C atom at an arbitrary point in space.
# The second line places the second C atom at the right distance, and therefore defines
# a direction: that of the C-C bond.
# The third line places a hydrogen, bonded to the first carbon atom and at the right angle
# with the C-C bond. Subsequent lines place the remaining H atoms in an analogous fashion.
# The last field in the last three lines is the dihedral angle formed by the planes defined by
# two C-C-H sequences. Since the molecule is planar, all of those dihedrals are either 0° or 180°
z_matrix = f"""
C 
C  1  {dCC}
H  1  {dCH}  2  {aHCC}
H  1  {dCH}  2  {aHCC}  3  180.
H  2  {dCH}  1  {aHCC}  3  0.
H  2  {dCH}  1  {aHCC}  3  180.
"""

In [5]:
def pyscf2ase(mol):
    "Small function that translates a molecule in PySCF format to an ASE atoms object."
    elements = mol.elements
    # These coordinates are in bohr and must be translated to angstrom.
    coords = mol.atom_coords() * ase.units.Bohr
    atoms = ase.Atoms(symbols=elements, positions=coords, pbc=False)
    return atoms

In [6]:
# Create the PySCF object
mol = pyscf.gto.M(atom=z_matrix, basis="sto-3g", unit="angstrom")

# And display it in an interactive widget to check that the positions make sense. We can measure
# distances and angles with the right mouse button.
view = nglview.show_ase(pyscf2ase(mol))
view.background="black"
display(view)

NGLWidget(background='black')

In [7]:
mol.elements

['C', 'C', 'H', 'H', 'H', 'H']

In [9]:
mol.atom_coords()

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.52467410e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.07770674e+00,  0.00000000e+00,  1.74427491e+00],
       [-1.07770674e+00,  2.13612069e-16, -1.74427491e+00],
       [ 3.60238085e+00,  0.00000000e+00,  1.74427491e+00],
       [ 3.60238085e+00, -2.13612069e-16, -1.74427491e+00]])

In [7]:
# Set up and launch the LDA calculation.
mf = pyscf.dft.RKS(mol)
mf.xc = "LDA"
mf.build()
mf.run()

converged SCF energy = -75.8649127653598


Calculate the KS eigenvalues and compare them to those you would obtain using the PBE. Plot the $\pi$-MO

In [8]:
# Get a quick summary of the results, including the energies we are interested in.
mf.analyze()
# Store the energies in a variable for laters use.
eigenvalues_LDA = mf.mo_energy.copy()

**** MO energy ****
MO #1   energy= -9.48502261457109  occ= 2
MO #2   energy= -9.48402058361088  occ= 2
MO #3   energy= -0.573220666812816 occ= 2
MO #4   energy= -0.41633770116344  occ= 2
MO #5   energy= -0.317427517461029 occ= 2
MO #6   energy= -0.260195167684734 occ= 2
MO #7   energy= -0.204696932993858 occ= 2
MO #8   energy= -0.117722875884236 occ= 2
MO #9   energy= 0.138305215985333  occ= 0
MO #10  energy= 0.409269369149414  occ= 0
MO #11  energy= 0.464900457413006  occ= 0
MO #12  energy= 0.4720422477334    occ= 0
MO #13  energy= 0.653503135183891  occ= 0
MO #14  energy= 0.691101475259256  occ= 0
 ** Mulliken atomic charges  **
charge of  0C =     -0.11640
charge of  1C =     -0.11640
charge of  2H =      0.05820
charge of  3H =      0.05820
charge of  4H =      0.05820
charge of  5H =      0.05820
Dipole moment(X, Y, Z, Debye):  0.00000, -0.00000,  0.00000


In [9]:
# Change the XC functional to PBE and redo the calculation.
mf.xc = "PBE"
mf.build()
mf.run()
mf.analyze()
eigenvalues_PBE = mf.mo_energy.copy()

converged SCF energy = -77.5081154672064
**** MO energy ****
MO #1   energy= -9.69768509540486  occ= 2
MO #2   energy= -9.69709910103633  occ= 2
MO #3   energy= -0.642534396248714 occ= 2
MO #4   energy= -0.483951168556139 occ= 2
MO #5   energy= -0.380621966685548 occ= 2
MO #6   energy= -0.324800163245933 occ= 2
MO #7   energy= -0.271714156214548 occ= 2
MO #8   energy= -0.177191887761333 occ= 2
MO #9   energy= 0.0785811184013614 occ= 0
MO #10  energy= 0.344047046722841  occ= 0
MO #11  energy= 0.396730139506371  occ= 0
MO #12  energy= 0.399444073884239  occ= 0
MO #13  energy= 0.590793216209897  occ= 0
MO #14  energy= 0.631060696346178  occ= 0
 ** Mulliken atomic charges  **
charge of  0C =     -0.10783
charge of  1C =     -0.10783
charge of  2H =      0.05392
charge of  3H =      0.05392
charge of  4H =      0.05392
charge of  5H =      0.05392
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -0.00000


In [10]:
# Print a list of the differences in energy.
print("MO#\tε_LDA\tε_PBE\tΔε")
for i, (e_LDA, e_PBE) in enumerate(zip(eigenvalues_LDA, eigenvalues_PBE)):
    print(f"{i + 1}\t{e_LDA:.3f}\t{e_PBE:.3f}\t{e_PBE - e_LDA:.3g}")


MO#	ε_LDA	ε_PBE	Δε
1	-9.485	-9.698	-0.213
2	-9.484	-9.697	-0.213
3	-0.573	-0.643	-0.0693
4	-0.416	-0.484	-0.0676
5	-0.317	-0.381	-0.0632
6	-0.260	-0.325	-0.0646
7	-0.205	-0.272	-0.067
8	-0.118	-0.177	-0.0595
9	0.138	0.079	-0.0597
10	0.409	0.344	-0.0652
11	0.465	0.397	-0.0682
12	0.472	0.399	-0.0726
13	0.654	0.591	-0.0627
14	0.691	0.631	-0.06


In [11]:
# Save each MO to a cube file.
for i in range(len(eigenvalues_PBE)):
    pyscf.tools.cubegen.orbital(mol, f"ethylene_MO_{i+1}_LDA.cube", mf.mo_coeff[:, i])

In [12]:
# Plot the π MO. It is easy to visualize these cube files directly outside of the notebook,
# but nglview also enables us to see them directly here.
view = nglview.show_ase(pyscf2ase(mol))
view.background = "black"
view.add_component("ethylene_MO_8.cube");
# view.component_0.clear()
view.component_1.clear()
# We use "sigma" to express our values as a fraction of the standard deviation of the data. 
view.component_1.add_surface(color="blue", isolevelType="sigma", isolevel=1., opacity=.6)
view.component_1.add_surface(color="red", isolevelType="sigma", isolevel=-1., opacity=.6)
display(view)

NGLWidget(background='black')

In [13]:
# Plot the π MO. It is easy to visualize these cube files directly outside of the notebook,
# but nglview also enables us to see them directly here.
view = nglview.show_ase(pyscf2ase(mol))
view.background = "black"
view.add_component("ethylene_MO_8_LDA.cube");
# view.component_0.clear()
view.component_1.clear()
# We use "sigma" to express our values as a fraction of the standard deviation of the data. 
view.component_1.add_surface(color="blue", isolevelType="sigma", isolevel=1., opacity=.6)
view.component_1.add_surface(color="red", isolevelType="sigma", isolevel=-1., opacity=.6)
display(view)

NGLWidget(background='black')